In [16]:
import recordlinkage
import pandas as pd

In [2]:
#Importing the data 
from recordlinkage.datasets import load_febrl4

In [186]:
df1, df2, true_links = load_febrl4(return_links=True)

In [188]:
df1.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-1070-org,michaela,neumann,8,stanley street,miami,winston hills,4223,nsw,19151111,5304218
rec-1016-org,courtney,painter,12,pinkerton circuit,bega flats,richlands,4560,vic,19161214,4066625
rec-4405-org,charles,green,38,salkauskas crescent,kela,dapto,4566,nsw,19480930,4365168
rec-1288-org,vanessa,parr,905,macquoid place,broadbridge manor,south grafton,2135,sa,19951119,9239102
rec-3585-org,mikayla,malloney,37,randwick road,avalind,hoppers crossing,4552,vic,19860208,7207688


In [189]:
df2.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-561-dup-0,elton,NaN,3,light setreet,pinehill,windermere,3212,vic,19651013,1551941
rec-2642-dup-0,mitchell,maxon,47,edkins street,lochaoair,north ryde,3355,nsw,19390212,8859999
rec-608-dup-0,NaN,white,72,lambrigg street,kelgoola,broadbeach waters,3159,vic,19620216,9731855
rec-3239-dup-0,elk i,menzies,1,lyster place,NaN,northwood,2585,vic,19980624,4970481
rec-2886-dup-0,NaN,garanggar,NaN,may maxwell crescent,springettst arcade,forest hill,2342,vic,19921016,1366884


### Data Preprocessing using Record Linkage.phonetic

In [190]:
from recordlinkage.preprocessing import  phonetic

In [191]:
print("\n Using Soundex algorithm four names refered to the same value")
Maya = pd.Series(["maya", "mya", "mayaa", "myaa"])
phonetic_maya = phonetic(Maya, "soundex")

for i in range(4):
    print(Maya[i] + " --> " + phonetic_maya[i])


 Using Soundex algorithm four names refered to the same value
maya --> M000
mya --> M000
mayaa --> M000
myaa --> M000


In [192]:
df1["phonetic_given_name"] = phonetic(df1["given_name"], "soundex")
df2["phonetic_given_name"] = phonetic(df2["given_name"], "soundex")
df1["phonetic_surname"] = phonetic(df1["surname"], "soundex")
df2["phonetic_surname"] = phonetic(df2["surname"], "soundex")

In [193]:
df1.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id,phonetic_given_name,phonetic_surname
rec_id,,,,,,,,,,,,
rec-1070-org,michaela,neumann,8,stanley street,miami,winston hills,4223,nsw,19151111,5304218,M240,N550
rec-1016-org,courtney,painter,12,pinkerton circuit,bega flats,richlands,4560,vic,19161214,4066625,C635,P536
rec-4405-org,charles,green,38,salkauskas crescent,kela,dapto,4566,nsw,19480930,4365168,C642,G650
rec-1288-org,vanessa,parr,905,macquoid place,broadbridge manor,south grafton,2135,sa,19951119,9239102,V520,P600
rec-3585-org,mikayla,malloney,37,randwick road,avalind,hoppers crossing,4552,vic,19860208,7207688,M240,M450


#### Checking for duplicate name entries  with same given name and surname

In [194]:
n1= df1.groupby(['phonetic_given_name', 'phonetic_surname']).count().reset_index()

In [195]:
n1[n1.given_name>1]
# Hence there are 280 pairs of people with same given name and surname

,phonetic_given_name,phonetic_surname,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
7,A100,F540,2,2,2,1,2,2,2,2,2,2
23,A100,R500,2,2,2,2,2,2,2,2,2,2
30,A124,P362,2,2,2,2,2,2,2,2,2,2
35,A164,M262,2,2,2,2,2,2,2,2,2,2
57,A242,M320,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...
4465,Z260,T610,2,2,2,2,1,2,2,2,2,2
4467,Z260,W300,2,2,2,2,1,2,2,2,1,2
4472,Z400,S314,2,2,2,2,1,2,2,2,2,2
4473,Z400,W300,2,2,2,2,2,2,2,2,2,2


#### Finding people  with phonetic_given_name= A100 and phonetic_surname= F540

In [196]:
df1[(df1['phonetic_given_name']== 'A100') & (df1['phonetic_surname']== 'F540')]

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id,phonetic_given_name,phonetic_surname
rec_id,,,,,,,,,,,,
rec-820-org,abbie,finlay,5,NaN,unt 1,mount warren park,4350,nsw,19730526,4857374,A100,F540
rec-220-org,abbey,finlay,14,lipman street,willowcrest,park holme,2680,vic,19711028,2407770,A100,F540


In [197]:
n2= df2.groupby(['phonetic_given_name', 'phonetic_surname']).count().reset_index()

In [198]:
n2[n2.given_name>1]
# Hence there are 200 pairs of people with same given name and surname

,phonetic_given_name,phonetic_surname,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
10,A100,G650,2,2,2,2,2,2,2,2,2,2
14,A100,M320,2,2,2,2,2,1,2,2,2,2
22,A100,R500,2,2,2,2,2,2,2,2,2,2
23,A100,S162,2,2,2,2,1,1,2,2,2,2
79,A350,F540,3,3,3,2,2,3,3,2,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...
4389,Z260,C462,2,2,2,1,2,2,2,2,2,2
4405,Z260,T610,2,2,1,2,2,2,2,2,2,2
4408,Z260,W300,2,2,2,2,1,2,2,2,2,2
4415,Z400,W300,2,2,1,2,2,2,2,2,2,2


#### Finding people  with phonetic_given_name= A100 and phonetic_surname= G650

In [199]:
df2[(df2['phonetic_given_name']== 'A100') & (df2['phonetic_surname']== 'G650')]

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id,phonetic_given_name,phonetic_surname
rec_id,,,,,,,,,,,,
rec-2174-dup-0,abbey,grewn,6,aspinall street,roseda le,mannum,3012,vic,19300314,5453075,A100,G650
rec-2078-dup-0,abbie,green,25,chisholm street,lazy acres,balwyn north,4413,aw,19630816,8182676,A100,G650


##### There are different type of index. If full index is selected then algorithm will generate possible combinations of record pairs. Hence, time complexity will be high. Hence, to match similar instances I have selected block index where record pairs agree on one or more rules. I am going to block index on "fullname"

In [247]:
indexer = recordlinkage.Index()
indexer.block('given_name')
similar_pairs = indexer.index(df1, df2)

print(similar_pairs )

MultiIndex([('rec-1070-org', 'rec-3024-dup-0'),
            ('rec-1070-org', 'rec-2371-dup-0'),
            ('rec-1070-org', 'rec-4652-dup-0'),
            ('rec-1070-org', 'rec-4795-dup-0'),
            ('rec-1070-org', 'rec-1314-dup-0'),
            ('rec-2371-org', 'rec-3024-dup-0'),
            ('rec-2371-org', 'rec-2371-dup-0'),
            ('rec-2371-org', 'rec-4652-dup-0'),
            ('rec-2371-org', 'rec-4795-dup-0'),
            ('rec-2371-org', 'rec-1314-dup-0'),
            ...
            ('rec-1413-org', 'rec-1413-dup-0'),
            ( 'rec-735-org',  'rec-735-dup-0'),
            ('rec-1361-org', 'rec-1361-dup-0'),
            ('rec-3090-org', 'rec-3090-dup-0'),
            ('rec-2571-org', 'rec-2571-dup-0'),
            ('rec-4528-org', 'rec-4528-dup-0'),
            ('rec-4887-org', 'rec-4887-dup-0'),
            ('rec-4350-org', 'rec-4350-dup-0'),
            ('rec-4569-org', 'rec-4569-dup-0'),
            ('rec-3125-org', 'rec-3125-dup-0')],
           names=['rec_

###### Next step is measuring similary between these pairs. In this case, I will use exact match on phonetic columns and similar match on strings  using jarowinkler algorithm and threshold set at 0.85. There are two algorithms that can be used to measure the similarity between pairs of strings: Jaro-Winkler and Levenshtein distance. Jaro-Winkler is a measure that takes into account the number of matching characters and any required transpositions between the two strings, and also gives more weight to prefixes that match. Levenshtein distance, on the other hand, counts the number of edit operations (insertions, deletions, or substitutions) needed to transform one string into another. Jaro-Winkler is generally a good choice when matching prefixes are more important, while Levenshtein distance is useful when allowing for a certain number of errors or differences between the strings. 

In [248]:
compare = recordlinkage.Compare()

compare.exact('given_name', 'given_name', label="given_name")
compare.exact('phonetic_given_name', 'phonetic_given_name', label='phonetic_given_name')
compare.exact('phonetic_surname', 'phonetic_surname', label='phonetic_surname')
compare.string('surname', 'surname', method='jarowinkler', threshold=0.85,label="surname")
compare.string('suburb', 'suburb', method='jarowinkler',label="suburb")
compare.string('state', 'state', method='jarowinkler',label="state")
compare.string('address_1', 'address_1', method='jarowinkler', label="address_1")
compare.string("soc_sec_id","soc_sec_id", method='jarowinkler',label="soc_sec_id")

features = compare.compute(similar_pairs, df1, df2)
features

given_name  phonetic_given_name  \
rec_id_1     rec_id_2                                          
rec-1070-org rec-3024-dup-0           1                    1   
             rec-2371-dup-0           1                    1   
             rec-4652-dup-0           1                    1   
             rec-4795-dup-0           1                    1   
             rec-1314-dup-0           1                    1   
...                                 ...                  ...   
rec-4528-org rec-4528-dup-0           1                    1   
rec-4887-org rec-4887-dup-0           1                    1   
rec-4350-org rec-4350-dup-0           1                    1   
rec-4569-org rec-4569-dup-0           1                    1   
rec-3125-org rec-3125-dup-0           1                    1   

                             phonetic_surname  surname    suburb  state  \
rec_id_1     rec_id_2                                                     
rec-1070-org rec-3024-dup-0                 0      0.0  0.584046    1.0   
             rec-2371-dup-0                 0      0.0  0.410256    0.0   
             rec-4652-dup-0                 0      0.0  0.515797    0.0   
             rec-4795-dup-0                 0      0.0  0.458689    1.0   
             rec-1314-dup-0                 0      0.0  0.313187    1.0   
...                                       ...      ...       ...    ...   
rec-4528-org rec-4528-dup-0                 1      1.0  1.000000    1.0   
rec-4887-org rec-4887-dup-0                 1      1.0  0.983333    1.0   
rec-4350-org rec-4350-dup-0                 1      1.0  1.000000    1.0   
rec-4569-org rec-4569-dup-0                 1      1.0  1.000000    1.0   
rec-3125-org rec-3125-dup-0                 1      1.0  0.424242    1.0   

                             address_1  soc_sec_id  
rec_id_1     rec_id_2                               
rec-1070-org rec-3024-dup-0   0.000000    0.523810  
             rec-2371-dup-0   0.715873    0.619048  
             rec-4652-dup-0   0.645604    0.523810  
             rec-4795-dup-0   0.552381    0.619048  
             rec-1314-dup-0   0.618254    0.619048  
...                                ...         ...  
rec-4528-org rec-4528-dup-0   0.985714    1.000000  
rec-4887-org rec-4887-dup-0   1.000000    1.000000  
rec-4350-org rec-4350-dup-0   1.000000    1.000000  
rec-4569-org rec-4569-dup-0   0.984615    0.909524  
rec-3125-org rec-3125-dup-0   1.000000    1.000000  

[77249 rows x 8 columns]

In [249]:
df1[df1.index=="rec-1070-org"]

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id,phonetic_given_name,phonetic_surname
rec_id,,,,,,,,,,,,
rec-1070-org,michaela,neumann,8,stanley street,miami,winston hills,4223,nsw,19151111,5304218,M240,N550


In [250]:
df2[df2.index=="rec-3024-dup-0"]

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id,phonetic_given_name,phonetic_surname
rec_id,,,,,,,,,,,,
rec-3024-dup-0,michaela,wilkns,2,NaN,robley house,jamdstown,3782,nsw,19390517,4477585,M240,W425


In [251]:
features.describe()

,given_name,phonetic_given_name,phonetic_surname,surname,suburb,state,address_1,soc_sec_id
count,77249.0,77249.0,77249.000000,77249.000000,77249.000000,77249.000000,77249.000000,77249.000000
mean,1.0,1.0,0.039573,0.044428,0.468918,0.268733,0.549879,0.517448
std,0.0,0.0,0.194956,0.206045,0.185887,0.435344,0.187851,0.171517
min,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.0,1.0,0.000000,0.000000,0.416667,0.000000,0.491865,0.428571
50%,1.0,1.0,0.000000,0.000000,0.477273,0.000000,0.557540,0.523810
75%,1.0,1.0,0.000000,0.000000,0.540476,0.822222,0.636508,0.619048
max,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Classification

Final step in the record linkage process is to classify each candidate pair as either a match or a non-match. One way to do this is to set a threshold based on the scores calculated in the previous steps, and classify any pairs that exceed the threshold as matches.

To evaluate the performance of the record linkage process,two metrics are used: precision and recall.

In [252]:
for i in range(7):
    matches = features[features.sum(axis=1) > i]
    pre = recordlinkage.precision(true_links, matches)
    rcl = recordlinkage.recall(true_links, matches)
    print("when score is " + str(i) + " precision is " + str(pre) + " and recall is " + str(rcl))

when score is 0 precision is 0.042550712630584214 and recall is 0.6574
when score is 1 precision is 0.042550712630584214 and recall is 0.6574
when score is 2 precision is 0.042558976616516045 and recall is 0.6574
when score is 3 precision is 0.044854737244306164 and recall is 0.6574
when score is 4 precision is 0.15818610696577287 and recall is 0.6572
when score is 5 precision is 0.8975206611570248 and recall is 0.6516
when score is 6 precision is 0.9771241830065359 and recall is 0.598


#### Hence at 5, precision is 89% and recall is 66%  There is always a tradeoff between precision and recall. Hence to balance both of them 5 is selected to get find out which records belongs to the same person 

In [255]:
recordlinkage.fscore(true_links, matches)

0.7550405561993047

#### Next is to find out which records belong to the same person

In [256]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

8.000000    867
7.991667      1
7.991304      1
7.990909      1
7.990476      1
           ... 
2.324074      1
2.305556      1
2.292023      1
2.285185      1
2.000000     15
Length: 54163, dtype: int64

In [257]:
matches = features[features.sum(axis=1) > 5]
matches

,,given_name,phonetic_given_name,phonetic_surname,surname,suburb,state,address_1,soc_sec_id
rec_id_1,rec_id_2,,,,,,,,
rec-2371-org,rec-2371-dup-0,1,1,1,1.0,1.000000,1.000000,1.000000,1.000000
rec-3024-org,rec-3024-dup-0,1,1,1,1.0,0.948148,1.000000,0.000000,1.000000
rec-4652-org,rec-4652-dup-0,1,1,1,1.0,0.541667,1.000000,0.952564,1.000000
rec-4795-org,rec-4795-dup-0,1,1,1,1.0,1.000000,1.000000,1.000000,1.000000
rec-1016-org,rec-1016-dup-0,1,1,1,1.0,1.000000,0.555556,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...
rec-4528-org,rec-4528-dup-0,1,1,1,1.0,1.000000,1.000000,0.985714,1.000000
rec-4887-org,rec-4887-dup-0,1,1,1,1.0,0.983333,1.000000,1.000000,1.000000
rec-4350-org,rec-4350-dup-0,1,1,1,1.0,1.000000,1.000000,1.000000,1.000000


In [260]:
df1[df1.index=="rec-4350-org"]

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id,phonetic_given_name,phonetic_surname
rec_id,,,,,,,,,,,,
rec-4350-org,aneka,milburn,267,wilkins street,clisby farm,kuraby,2715,qld,19171108,4533360,A520,M416


In [261]:
df2[df2.index=="rec-4350-dup-0"]

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id,phonetic_given_name,phonetic_surname
rec_id,,,,,,,,,,,,
rec-4350-dup-0,aneka,milburn,103,wilkins street,clisbyfarm,kuraby,2715,qld,19171108,4533360,A520,M416


## Classfying using ML Algorithms

In [262]:
clasifier = recordlinkage.LogisticRegressionClassifier()
clasifier.fit(features, true_links)
predictions = clasifier.predict(features)

print("precision: " + str(recordlinkage.precision(true_links, predictions)))
print("recall: " + str(recordlinkage.recall(true_links, predictions)))

precision: 0.9947948560930803
recall: 0.6498


#### Analysing False Positives

In [263]:
print("Number of FP instances")
recordlinkage.false_positives(true_links, predictions  ) 

Number of FP instances


17

Out of 100,000 records there are only 17 False Positive cases. 

In [264]:
false_positives = predictions.difference(true_links)
false_positives

MultiIndex([('rec-1066-org', 'rec-2167-dup-0'),
            ('rec-1546-org', 'rec-3587-dup-0'),
            ('rec-1591-org',  'rec-361-dup-0'),
            ( 'rec-166-org', 'rec-4758-dup-0'),
            ('rec-1962-org', 'rec-4130-dup-0'),
            ('rec-2097-org', 'rec-4124-dup-0'),
            ('rec-2167-org', 'rec-1066-dup-0'),
            ('rec-2587-org', 'rec-4470-dup-0'),
            ('rec-3089-org',  'rec-272-dup-0'),
            ('rec-3569-org', 'rec-3829-dup-0'),
            ('rec-3587-org', 'rec-1546-dup-0'),
            ( 'rec-394-org', 'rec-4790-dup-0'),
            ('rec-4014-org', 'rec-2083-dup-0'),
            ('rec-4130-org', 'rec-1962-dup-0'),
            ('rec-4470-org', 'rec-2587-dup-0'),
            ('rec-4484-org',  'rec-969-dup-0'),
            ('rec-4984-org', 'rec-4864-dup-0')],
           )

In [265]:
display(dfA[dfA.index == false_positives[0][0]])
display(dfB[dfB.index == false_positives[0][1]])

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-1066-org,sarah,campbell,9,clavert place,merricroft,mount eliza,5280,qld,19410302,9870428


,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-2167-dup-0,sarah,campbell,6,loddon street,NaN,miasmi,3227,qld,19321221,5872484


#### Analysing False Negatives

In [266]:
print("Number of FN instances")
recordlinkage.false_negatives(true_links, predictions  ) 



Number of FN instances


1751

In [267]:
false_negatives = true_links.difference(predictions)
false_negatives

MultiIndex([(  'rec-10-org',   'rec-10-dup-0'),
            ('rec-1003-org', 'rec-1003-dup-0'),
            ('rec-1007-org', 'rec-1007-dup-0'),
            ('rec-1009-org', 'rec-1009-dup-0'),
            ('rec-1010-org', 'rec-1010-dup-0'),
            ('rec-1011-org', 'rec-1011-dup-0'),
            ('rec-1015-org', 'rec-1015-dup-0'),
            ('rec-1017-org', 'rec-1017-dup-0'),
            ('rec-1018-org', 'rec-1018-dup-0'),
            ('rec-1019-org', 'rec-1019-dup-0'),
            ...
            (  'rec-98-org',   'rec-98-dup-0'),
            ( 'rec-981-org',  'rec-981-dup-0'),
            ( 'rec-982-org',  'rec-982-dup-0'),
            ( 'rec-983-org',  'rec-983-dup-0'),
            ( 'rec-987-org',  'rec-987-dup-0'),
            ( 'rec-989-org',  'rec-989-dup-0'),
            (  'rec-99-org',   'rec-99-dup-0'),
            ( 'rec-990-org',  'rec-990-dup-0'),
            ( 'rec-992-org',  'rec-992-dup-0'),
            ( 'rec-994-org',  'rec-994-dup-0')],
           length=1751)

In [268]:
display(dfA[dfA.index == false_negatives[0][0]])
display(dfB[dfB.index == false_negatives[0][1]])

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-10-org,lachlan,reid,5,carrington road,legacy vlge,yagoona,2464,nsw,19500531,3232033


,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-10-dup-0,lachlnn,reid,5,carrington road,legacy vlge,yagoona,2446,nsw,19500531,3232033
